In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import ast
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torch.nn as nn
from torch.cuda.amp import GradScaler
from torchvision.models.resnet import resnet50
from torchvision.models.detection.faster_rcnn import FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FasterRCNN, FasterRCNN_ResNet50_FPN_V2_Weights, FastRCNNPredictor, FastRCNNConvFCHead
from torchvision.models.detection.backbone_utils import _resnet_fpn_extractor
from torchvision.models.detection.rpn import AnchorGenerator, RPNHead

In [ ]:

def label_encoder(labels):
  encode_label = {"RBC": 1, "trophozoite": 3, "difficult": 4, "ring": 5, "schizont": 6,
                "gametocyte": 7, "leukocyte": 2}
  return [encode_label[l] for l in labels]

def get_transform():
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, root_dir, annotation_path, transforms, label_encoder):
    self.root_dir = root_dir
    self.annotations = self.read_ann(annotation_path)
    self.transforms = transforms
    self.encode_label = label_encoder

  def read_ann(self, ann_path):
    df = pd.read_csv(ann_path)
    df['boxes'] = df['boxes'].apply(ast.literal_eval)
    df['label'] = df['label'].apply(ast.literal_eval)
    df['area'] = df['area'].apply(ast.literal_eval)
    return df

  def get_target(self, idx):
    row = self.annotations.iloc[idx]
    boxes = row['boxes']
    labels = self.encode_label(row['label'])
    area = row['area']
    iscrowd = torch.zeros((len(boxes),), dtype=torch.int64)
    target = {
        'boxes': torch.tensor(boxes, dtype=torch.int),
        'labels': torch.tensor(labels),
        'area': torch.tensor(area, dtype=torch.float32),
        'image_id': torch.tensor([idx]),
        'iscrowd': iscrowd
    }
    return target

  def __getitem__(self, idx):
    record = self.annotations.iloc[idx]
    filepath = os.path.join(self.root_dir, record['pathname'][1:])
    target = self.get_target(idx)
    image = Image.open(filepath).convert('RGB')

    if self.transforms is not None:
        image = self.transforms(image)
    return image, target

  def __len__(self):
    return len(self.annotations)

In [ ]:
%%shell

pip install cython
# Install pycocotools, the version by default in Colab
# has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'


# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

In [ ]:
def custom_anchorgen():
    anchor_sizes = ((32,), (64,), (80,), (94,), (128,))
    aspect_ratios = ((0.5, 1.0, 2.0),) * len(anchor_sizes)
    return AnchorGenerator(anchor_sizes, aspect_ratios)

def get_fastercnn_v1(num_classes):
  anchor_generator = custom_anchorgen()
  kwarg = {
      'image_mean': [0.7205, 0.7203, 0.7649],
      'image_std': [0.2545, 0.2599, 0.2037],
      # RPN parameters
      'rpn_anchor_generator': anchor_generator,
  }
  model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT, kwarg=kwarg)
  in_features = model.roi_heads.box_predictor.cls_score.in_features
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
  return model

In [ ]:
from engine import evaluate
import utils
from torchvision import transforms as T

In [ ]:
root_dir = "/malaria"
train_path = "/malaria/train.csv"
test_path = "/malaria/test.csv"

train_dataset = CustomDataset(root_dir=root_dir, annotation_path=train_path, transforms=get_transform(), label_encoder=label_encoder)
test_dataset = CustomDataset(root_dir=root_dir, annotation_path=test_path, transforms=get_transform(), label_encoder=label_encoder)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=6, shuffle=True, num_workers=2,
    collate_fn=utils.collate_fn)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=2, shuffle=False, num_workers=2,
    collate_fn=utils.collate_fn)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#number of classes in our dataset
num_classes = 8

# get the model using our helper function
model = get_fastercnn_v1(num_classes)

total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")

total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.001,
                            momentum=0.9, weight_decay=0.0001, nesterov=False)

scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.85)

scaler = GradScaler()

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 138MB/s]


41,329,911 total parameters.
41,107,511 training parameters.


In [ ]:
import math
import sys
import time

import torch
import torchvision.models.detection.mask_rcnn
import utils
from coco_eval import CocoEvaluator
from coco_utils import get_coco_api_from_dataset


def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq, scaler=None):
    model.train()
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter("lr", utils.SmoothedValue(window_size=1, fmt="{value:.6f}"))
    header = f"Epoch: [{epoch}]"

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1.0 / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = torch.optim.lr_scheduler.LinearLR(
            optimizer, start_factor=warmup_factor, total_iters=warmup_iters
        )

    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        with torch.cuda.amp.autocast(enabled=scaler is not None):
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = utils.reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping training")
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        if scaler is not None:
            scaler.scale(losses).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            losses.backward()
            optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

    return metric_logger

In [ ]:
num_epochs = 12

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, train_loader, device, epoch, scaler=scaler, print_freq=20)
    # update the learning rate
    if epoch >= 1:
      scheduler.step()
    # evaluate on the test dataset
    evaluate(model, test_loader, device=device)

Epoch: [0]  [  0/202]  eta: 0:08:07  lr: 0.000006  loss: 0.4665 (0.4665)  loss_classifier: 0.1845 (0.1845)  loss_box_reg: 0.2597 (0.2597)  loss_objectness: 0.0020 (0.0020)  loss_rpn_box_reg: 0.0203 (0.0203)  time: 2.4139  data: 1.3775  max mem: 7041
Epoch: [0]  [ 20/202]  eta: 0:03:13  lr: 0.000105  loss: 0.4274 (0.4304)  loss_classifier: 0.1839 (0.1799)  loss_box_reg: 0.2264 (0.2262)  loss_objectness: 0.0057 (0.0058)  loss_rpn_box_reg: 0.0184 (0.0185)  time: 0.9955  data: 0.0854  max mem: 7041
Epoch: [0]  [ 40/202]  eta: 0:02:41  lr: 0.000205  loss: 0.4216 (0.4300)  loss_classifier: 0.1740 (0.1789)  loss_box_reg: 0.2252 (0.2258)  loss_objectness: 0.0051 (0.0057)  loss_rpn_box_reg: 0.0190 (0.0196)  time: 0.9323  data: 0.0590  max mem: 7041
Epoch: [0]  [ 60/202]  eta: 0:02:19  lr: 0.000304  loss: 0.4025 (0.4282)  loss_classifier: 0.1726 (0.1777)  loss_box_reg: 0.2076 (0.2245)  loss_objectness: 0.0054 (0.0059)  loss_rpn_box_reg: 0.0197 (0.0201)  time: 0.9432  data: 0.0658  max mem: 7041


In [ ]:
torch.save(model.state_dict(), "model_f.pt")